In [ ]:
import psycopg2
import pandas as pd
halyk = pd.read_excel('БЦК.xlsx')
halyk_str = halyk['БИН'].astype(str)

bins_list = halyk_str.to_list()
new_bins_list = []

for item in bins_list:
    if len(item) != 12:
        item = '0'*(12-len(item))+item
    new_bins_list.append(item)


new_bins_list
min_length =13
for item in new_bins_list:
    if len(item)<min_length:
        min_length = len(item)
min_length
halyk['БИН'] = pd.DataFrame(new_bins_list)
conn = psycopg2.connect(host=host, port=port, database=database, user=user, password=password)


cursor = conn.cursor()
query = f'''
        
        SELECT bin, city, change_date
        FROM (
            SELECT bin, city, change_date,
                   ROW_NUMBER() OVER (PARTITION BY bin ORDER BY change_date DESC) AS rn
            FROM (
                SELECT DISTINCT ON (bin) bin, city, change_date
                FROM dwh_rb.stat_gov AS act
                WHERE act.bin IN {tuple(new_bins_list)}

                UNION

                SELECT DISTINCT ON (bin) bin, city, change_date
                FROM dwh_rb.stat_gov_hst AS hist
                WHERE hist.bin IN {tuple(new_bins_list)}

                ORDER BY bin, change_date DESC
            ) AS combined_data
        ) AS ranked_data
        WHERE rn = 1;
        '''


cursor.execute(query)

results = cursor.fetchall()
df = pd.DataFrame(results, columns=['БИН', 'city', 'change date'])
df['city'] = df['city'].str.split(',').str[0]
total_df = pd.merge(halyk, df, on='БИН', how='left')
total_df = total_df.drop(['change date'], axis=1)

cursor.close()
conn.close()


print(df)